In [1]:
from minitrade.backtest import Strategy

class MyStrategy(Strategy):
    def init(self):
        self.roc = self.I(self.data.ta.roc(10), name='ROC')

    def next(self):
        self.alloc.assume_zero()
        roc = self.roc.df.iloc[-1]
        (self.alloc.bucket['equity']
            .append(roc.sort_values(ascending=False), roc > 0)
            .trim(3)
            .weight_explicitly(1/3)
            .apply())
        self.rebalance(cash_reserve=0.01)

2025-05-04 21:05:56.761 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [3]:
from minitrade.datasource import QuoteSource
from minitrade.backtest import Backtest

yahoo = QuoteSource.get_source('Yahoo')
data = yahoo.daily_bar('MMM,AXP,AAPL,BA,CVX', start='2024-01-01')
bt = Backtest(data, MyStrategy, cash=10000)
bt.run()

YF deprecation warning: set proxy via new config function: yf.set_config(proxy=proxy)


Start                                                   2024-01-02 00:00:00
End                                                     2025-05-02 00:00:00
Duration                                                  486 days 00:00:00
Exposure Time [%]                                                 91.641791
Equity Final [$]                                               11422.900796
Equity Peak [$]                                                12060.686267
Return [%]                                                        14.229008
Buy & Hold Return [%]                                             26.779314
Return (Ann.) [%]                                                 10.866142
Volatility (Ann.) [%]                                             17.558447
Sharpe Ratio                                                       0.618856
Sortino Ratio                                                      0.911516
Calmar Ratio                                                       0.732289
Max. Drawdow

In [4]:
bt.plot(plot_allocation=True)

GridPlot(id='p1526', ...)